In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
nightstDF = pd.read_csv('./project/night_stand.csv', index_col=0)
nightstDF['image'] 

0        ./static/img/bed/night_stand/night_stand1.png
1        ./static/img/bed/night_stand/night_stand2.png
2        ./static/img/bed/night_stand/night_stand3.png
3        ./static/img/bed/night_stand/night_stand4.png
4        ./static/img/bed/night_stand/night_stand5.png
                            ...                       
97      ./static/img/bed/night_stand/night_stand98.png
98      ./static/img/bed/night_stand/night_stand99.png
99     ./static/img/bed/night_stand/night_stand100.png
100    ./static/img/bed/night_stand/night_stand101.png
101    ./static/img/bed/night_stand/night_stand102.png
Name: image, Length: 102, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(nightstDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

nightstDF['nouns'] = box

100%|██████████| 102/102 [00:16<00:00,  6.18it/s]


In [5]:
nightstDF.head(1)

,brand,item,price,image,link,review,nouns
0,제이픽스,원목 사이드테이블 협탁 (강화유리선반) 4color,"18,800",./static/img/bed/night_stand/night_stand1.png,https://ohou.se/productions/244188/selling?aff...,"['전체적으로 나온 사진이 없어서 이렇게 첨부해요,,ㅜ 원룸에서 사용하려고 구매했고...",전체 사진 첨부 원룸 사용 구매 여성 혼자 뚝딱 조립 사용 가격 가성 비 것 생각 ...


In [6]:
# 중간 저장
nightstDF.to_csv('nightstDF.csv')

In [7]:
nightstDF = pd.read_csv('./nightstDF.csv', index_col=0)
nightstDF.head(1)

,brand,item,price,image,link,review,nouns
0,제이픽스,원목 사이드테이블 협탁 (강화유리선반) 4color,"18,800",./static/img/bed/night_stand/night_stand1.png,https://ohou.se/productions/244188/selling?aff...,"['전체적으로 나온 사진이 없어서 이렇게 첨부해요,,ㅜ 원룸에서 사용하려고 구매했고...",전체 사진 첨부 원룸 사용 구매 여성 혼자 뚝딱 조립 사용 가격 가성 비 것 생각 ...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(nightstDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_nightst = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_nightst

,가가,가감,가게,가겍,가격,가경,가공,가교,가구,가구라,...,흰협탁,히든,히힛,힐링,힐수,힘껏,힘등,힘뽝,힘안,힙해
0,0,0,0,0,21,0,0,0,5,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,10,0,0,0,5,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,11,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,24,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,9,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,10,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,18,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
100,0,0,0,0,12,0,0,0,10,0,...,0,0,0,0,1,0,0,0,0,0


In [9]:
# dtm 파일 저장
dtm_nightst.to_csv('dtm_nightst.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

nightstDF = pd.read_csv('./nightstDF.csv', index_col=0)
dtm_nightst = pd.read_csv('./dtm_nightst.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(nightstDF['brand'])
item_list = np.array(nightstDF['item'])
price_list = np.array(nightstDF['price'])
image_list = np.array(nightstDF['image'])

result_cosine = make_cosine(dtm_nightst, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

올쏘퍼니처 (오늘의딜) 케인 라운드 침대 협탁 소파 사이드 테이블 3colors 29,900 ./static/img/bed/night_stand/night_stand15.png


In [12]:
def find_nightstand(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    nightstDF = pd.read_csv('./nightstDF.csv', index_col=0)
    dtm_nightst = pd.read_csv('./dtm_nightst.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(nightstDF['brand'])
    item_list = np.array(nightstDF['item'])
    price_list = np.array(nightstDF['price'])
    image_list = np.array(nightstDF['image'])

    result_cosine = make_cosine(dtm_nightst, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]